In [1]:
#from google.colab import drive

#drive.mount('/content/drive')
!mkdir dataset
!pip install gdown
!gdown 1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
!mv refcocog.tar.gz ./dataset/

mkdir: cannot create directory ‘dataset’: File exists
Downloading...
From (uriginal): https://drive.google.com/uc?id=1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
From (redirected): https://drive.google.com/uc?id=1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq&confirm=t&uuid=7ce04594-263c-4aa6-90de-6d4fef7f2f35
To: /home/disi/.jupyter/refcocog.tar.gz
100%|███████████████████████████████████████| 13.5G/13.5G [01:25<00:00, 158MB/s]


In [2]:
!tar -xf dataset/refcocog.tar.gz -C dataset
!ls dataset


refcocog  refcocog.tar.gz


In [1]:
!pip install --upgrade git+https://github.com/openai/CLIP.git

from clip import clip

model, preprocess = clip.load("RN50")

model = model.float()

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-jnk0akfq
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-jnk0akfq
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369522 sha256=0d20b1fc8f0d87537ecf6388ebfec1990b42113f19295c7d9bb1d125ed12c81a
  Stored in directory: /tmp/pip-ephem-wheel-cache-__d3bwmo/wheels/ab/4f/3a/5e51521b55997aa6f0690e095c08824219753128ce8d9969a3
Successfully built clip
  Attempting uninstall: clip
    Found existing installation: clip 1.0
    Uninstalling clip-1.0:
      Successfully uninstalled clip-1.0


In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:<1024>"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"
os.environ["TORCH_USE_CUDA_DSA"]="1"

In [3]:
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision import ops
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from posixpath import split
import json
import tarfile
import io
import pickle
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt




# import some common libraries
import cv2
#from google.colab.patches import cv2_imshow

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.autograd.set_detect_anomaly(True)
data = {}


In [4]:
def preProcess_datasets(data_dir):
  train_dataset = {}
  test_dataset = {}
  f = open(f'{data_dir}/annotations/refs(umd).p', 'rb')
  data['refs'] = pickle.load(f)
  instances_file = os.path.join(f'{data_dir}/annotations/instances.json')
  instances = json.load(open(instances_file, 'r'))
  data['images'] = instances['images']
  data['annotations'] = instances['annotations']

  c=0
  d=0
  for key,val in enumerate(data['refs']):
    for v in data['images']:
      if val['image_id'] == v['id']:
        fname = os.path.join(data_dir+'/images/', v['file_name'])
        if os.path.exists(fname):
          if val['split'] == 'train' or val['split'] == 'val':
            train_dataset[c] = {}
            train_dataset[c]['image'] = v['file_name']
            train_dataset[c]['captions'] = [t['raw'] for t in val['sentences']]
            train_dataset[c]['id'] = val['image_id']
            break
          else:
            test_dataset[d] = {}
            test_dataset[d]['image'] = v['file_name']
            test_dataset[d]['captions'] = [t['raw'] for t in val['sentences']]
            test_dataset[d]['id'] = val['image_id']
            break
    for ann in data['annotations']:
      if val['ann_id'] == ann['id']:
        if val['split'] == 'train' or val['split'] == 'val':
          train_dataset[c]['bbox'] = ann['bbox']
          c+=1
          break
        else:
          test_dataset[d]['bbox'] = ann['bbox']
          d+=1
          break
    if c>=5000:
      break

  return train_dataset, test_dataset


def resize_bounding_box(bbox, original_size, new_size):
  x_min, y_min, x_max, y_max = bbox
  orig_width, orig_height = original_size
  new_width, new_height = new_size

  # Calculate the scaling factors
  width_ratio = new_width / orig_width
  height_ratio = new_height / orig_height

  # Resize the bounding box coordinates
  new_x_min = int(x_min * width_ratio)
  new_y_min = int(y_min * height_ratio)
  new_x_max = int(x_max * width_ratio)
  new_y_max = int(y_max * height_ratio)

  return new_x_min, new_y_min, new_x_max, new_y_max

def xywh_to_xyxy(boxes):
  xmin = boxes[0]
  ymin = boxes[1]
  w = boxes[2]
  h = boxes[3]
  xmax = xmin + w
  ymax = ymin + h
  return [xmin, ymin, xmax, ymax]

In [5]:
class RefCOCOgDataset(Dataset):
    def __init__(self, dataset, transform, target_transform, data_dir='dataset/refcocog'):
        super(RefCOCOgDataset, self).__init__()
        self.data_dir = data_dir
        self.transform = transform
        self.target_transform = target_transform
        self.dataset = dataset



    def __getitem__(self, idx):
        data_item = {}
        fname = os.path.join(self.data_dir+'/images/', self.dataset[idx]['image'])
        image = Image.open(fname).convert('RGB')
        shape_0 = list(image.size)
        if self.transform:
            data_item['image'] = self.transform(image)
        if self.target_transform:
            data_item['captions'] = clip.tokenize([sent for desc in self.dataset[idx]['captions'] for sent in desc])
            bb = xywh_to_xyxy(self.dataset[idx]['bbox'])
            bbox = resize_bounding_box(bb, shape_0, [224,224])
            data_item['bbox'] = torch.tensor(bbox)
        return data_item


    def __len__(self):
        return len(self.dataset)



In [6]:
def get_cost_function():
  cost_function = torch.nn.SmoothL1Loss()
  return cost_function

def get_iou_torch(ground_truth, pred):
  # Coordinates of the area of intersection.

  ix1 = torch.max(ground_truth[0][0], pred[0][0])
  iy1 = torch.max(ground_truth[0][1], pred[0][1])
  ix2 = torch.min(ground_truth[0][2], pred[0][2])
  iy2 = torch.min(ground_truth[0][3], pred[0][3])

  # Intersection height and width.
  i_height = torch.max(iy2 - iy1 + 1, torch.tensor(0.))
  i_width = torch.max(ix2 - ix1 + 1, torch.tensor(0.))

  area_of_intersection = i_height * i_width

  # Ground Truth dimensions.
  gt_height = ground_truth[0][3] - ground_truth[0][1] + 1
  gt_width = ground_truth[0][2] - ground_truth[0][0] + 1

  # Prediction dimensions.
  pd_height = pred[0][3] - pred[0][1] + 1
  pd_width = pred[0][2] - pred[0][0] + 1

  area_of_union = gt_height * gt_width + pd_height * pd_width - area_of_intersection

  iou = (area_of_intersection + 1e-5) / (area_of_union + 1e-5) # Adding a small epsilon to avoid division by zero
  #average_iou = torch.mean(iou)

  return iou.detach().cpu().numpy()



In [7]:
def get_optimizer(model, lr, wd, momentum):
  optimizer = torch.optim.SGD([
      {'params': model.parameters(), 'lr': lr}
  ], lr=lr, weight_decay=wd, momentum=momentum)

  #optimizer = torch.optim.Adam(model.parameters(), lr=lr)

  return optimizer

In [8]:
def training_step(net, data_loader, optimizer, cost_function, device='cuda:0'):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0
    batch_ious = []
    # set the network to training mode
    net = net.float()
    net.train()
    accumulation_steps = 2
    clip_value = 1.0
  # iterate over the training set
    for batch_idx, batch in enumerate(data_loader):

      #load data into GPU
      batch['image'] = batch['image'].to(device)
      batch['captions'] = batch['captions'].to(device)
      target_bbox = batch['bbox'].to(device)
      # forward pass
      out_bbox = net(batch['image'], batch['captions'])
      out_bbox = out_bbox.to(device)
      # loss computation
      loss = cost_function(out_bbox, target_bbox)
      # fetch prediction and loss value
      # backward pass
      loss.requires_grad = True
      loss.backward()
        
      if (batch_idx + 1) % accumulation_steps == 0:
          # parameters update
          optimizer.step()
          # gradients reset
          optimizer.zero_grad()
          torch.nn.utils.clip_grad_norm_(net.parameters(), clip_value)
      # fetch prediction and loss value
      samples += batch['bbox'].shape[0]
        
      cumulative_loss += loss.item()

      iou_accuracy = torchvision.ops.box_iou(target_bbox, out_bbox).detach().cpu().numpy().sum().item()

      batch_ious.append(iou_accuracy)
    if batch_idx % accumulation_steps != 0:
        # parameters update
        optimizer.step()
        # gradients reset
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(net.parameters(), clip_value)
      # Calculate the average IoU across all batches
    if batch_ious:
        # Calculate the average IoU across all batches
        cumulative_accuracy = np.mean(batch_ious)
    else:
        cumulative_accuracy = 0.0 # Set to zero if there are no elements


    return cumulative_loss/samples, cumulative_accuracy

def test_step(net, data_loader, cost_function, device='cuda:0'):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0
    batch_ious = []

    net = net.float()
    # set the network to evaluation mode
    net.eval()

    # disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    with torch.no_grad():
        # iterate over the test set

      for batch_idx, batch in enumerate(data_loader):

        #load data into GPU
        batch['image'] = batch['image'].to(device)
        batch['captions'] = batch['captions'].to(device)
        target_bbox = batch['bbox'].to(device)

        # forward pass
        out_bbox = net(batch['image'], batch['captions'])
        out_bbox = out_bbox.to(device)
        
        # loss computation on GPU
        loss = cost_function(out_bbox, target_bbox)

        # accumulate the loss on the CPU
        cumulative_loss += loss.item()

        samples += batch['bbox'].shape[0]

        iou_accuracy = torchvision.ops.box_iou(target_bbox, out_bbox).detach().cpu().numpy().sum().item()
        batch_ious.append(iou_accuracy)

        # Calculate the average IoU across all batches
        if batch_ious:
            cumulative_accuracy = np.mean(batch_ious)
        else:
            cumulative_accuracy = 0.0  # Set to zero if there are no elements

    # Move the final cumulative loss back to the GPU
    #cumulative_loss = torch.tensor(cumulative_loss).to(device)

    return cumulative_loss / samples, cumulative_accuracy

In [9]:
def _make_grid(input):
    # Get the width and height of the output feature map
    _, width, height = input.size()
    # Determine the size of each grid cell
    grid_size = height // 16
    grid_cells = []
    grid_coordinates = []
    for i in range(16):
        for j in range(16):
            # Calculate the coordinates for the current grid cell
            x1 = i * grid_size
            y1 = j * grid_size
            x2 = x1 + grid_size
            y2 = y1 + grid_size
            # Extract the region corresponding to the grid cell
            grid_cell = input[:, x1:x2, y1:y2]
            grid_coordinates.append(torch.tensor([float(x1),float(y1),float(x2),float(y2)]))
            grid_cells.append(grid_cell)
    grid_cells = torch.stack(grid_cells)
    grid_coordinates = torch.stack(grid_coordinates)
    return grid_cells, grid_coordinates


class Grid(torch.nn.Module):
    def __init__(self):
        super(Grid, self).__init__()

    def forward(self, x):
        res = []
        for img in x:
            res.append(_make_grid(img))
        return res


class CustomCLIP(torch.nn.Module):
  def __init__(self, model, num_classes: int = 4, bias=False):
    super().__init__()
    self.model = model
    self.num_classes = num_classes
    self.encoder = self.model.visual.float()
    self.conv1 = self.encoder.conv1
    self.bn1 = self.encoder.bn1
    self.relu1 = self.encoder.relu1
    self.avgpool = self.encoder.avgpool
    self.fc = torch.nn.Linear(512, self.num_classes, bias=bias)
    self.relu = torch.nn.ReLU()


    self.text_encoder = self.encode_text

    self.grid = Grid()

    # add a bottleneck
    self.image_encoder = torch.nn.Sequential(
      self.conv1,
      self.bn1,
      self.relu1,
      self.avgpool,
      torch.nn.Flatten(),
      torch.nn.Linear(288, self.num_classes, bias=bias),
    )


  def encode_text(self, text):
      x = self.model.token_embedding(text)
      x = self.fc(x)
      x = x[:, :, :, :4]
      return x

  def forward(self, img, cap):
    grids = self.grid(img)
    aspect_ratio = 224 / 224
    target_width = 224 // 16
    target_height = int(target_width / aspect_ratio)
    similar = []
    for idx, (gd, _) in enumerate(grids):
      x_ = []
      for g in gd:
          x = F.interpolate(g.unsqueeze(0), size=(target_width, target_height), mode='bilinear', align_corners=False)
          x = self.image_encoder(x)
          x /= x.norm(dim=-1, keepdim=True)
          x_.append(x)
      #with torch.no_grad():
      y = self.text_encoder(cap[idx].unsqueeze(0))
      y /= y.norm(dim=-1, keepdim=True)


      im_ = torch.cat(x_)

      target_len = y.size(2) - im_.size(0)
      im_ = F.pad(im_, (0,0,0,target_len), value=0)


      similarity_scores = torch.nn.functional.cosine_similarity(im_, y, dim=-1)
      max_values, max_indices = torch.max(similarity_scores, dim=1)
      boxes = grids[idx][1]
      selected_box = boxes[max_indices.to(boxes.device)]

      similar.append(selected_box)

    out_bbox = torch.cat(similar)
    out_bbox = out_bbox[:, 0, :]

    return out_bbox

In [10]:
def pad_sequence(batch):
  return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0)
def my_collate_fn(batch):

  return {
      'image': pad_sequence([x['image'] for x in batch]),
      'captions': pad_sequence([x['captions'] for x in batch]),
      'bbox': pad_sequence([x['bbox'] for x in batch])
      }


def get_data(data_dir, batch_size, test_batch_size, transform=True, target_transform=False):


    if transform:
        # convert the PIL images to Tensors
        transform = T.Compose([
            T.Resize((224, 224)),
            T.RandomCrop(size=224),
            T.RandomHorizontalFlip(),
            T.RandomRotation(degrees=30),
            T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    else:
          # prepare data transformations and then combine them sequentially
        transform = None
    if target_transform:
        target_transform = T.Compose([
                                 lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)])
    else:
        target_transform = None

  # load data
    train_dataset, test_dataset = preProcess_datasets(data_dir)

    full_training_data = RefCOCOgDataset(dataset=train_dataset, transform=transform, target_transform=target_transform, data_dir=data_dir)


    global test_data
    test_data = RefCOCOgDataset(dataset=test_dataset, transform=transform, target_transform=target_transform, data_dir=data_dir)
    num_samples = len(full_training_data)

    training_samples = int(num_samples * 0.8 + 1)
    validation_samples = num_samples - training_samples

    training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])
    print(len(training_data))

    print(len(validation_data))
    print(len(test_data))
  # initialize dataloaders_collate
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=2, collate_fn=my_collate_fn)
    val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False, num_workers=2, collate_fn=my_collate_fn)
    test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=2, collate_fn=my_collate_fn)

    return train_loader, val_loader, test_loader


In [11]:
# tensorboard logging utilities
def log_values(writer, step, loss, accuracy, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)
  writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

In [12]:
# main funcition
def main(
      root='/content/dataset/refcocog/',
      data_dir='dataset/refcocog',
      batch_size=64,
      test_batch_size=224,
      num_classes=4,
      learning_rate=0.1,
      weight_decay=1e-5,
      momentum=0.9,
      epochs=10
      #epochs=3
    ):
  writer = SummaryWriter(log_dir="runs/exp1")
  #global test_loader
  # train clip on zero-shot learning and instantiates dataloaders
  train_loader, val_loader, test_loader = get_data(data_dir=data_dir, batch_size=batch_size, test_batch_size=test_batch_size, transform=True, target_transform=True)


  # instantiate the network and move it to the chosen device (GPU)
  
  global modified_model
  modified_model = CustomCLIP(model=model, num_classes=num_classes).to(device)

  # instantiate the optimizer
  optimizer = get_optimizer(modified_model, learning_rate, weight_decay, momentum)

  # define the cost function
  cost_function = get_cost_function()

  torch.cuda.empty_cache()
  # computes evaluation results before training
  print('Before training:')
  train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
  torch.cuda.empty_cache()
  val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
  torch.cuda.empty_cache()
  test_loss, test_accuracy = test_step(modified_model, test_loader, cost_function)
  torch.cuda.empty_cache()
  
  # log to TensorBoard
  log_values(writer, -1, train_loss, train_accuracy, "train")
  log_values(writer, -1, val_loss, val_accuracy, "validation")
  log_values(writer, -1, test_loss, test_accuracy, "test")

  print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')
  torch.cuda.empty_cache()
  # for each epoch, train the network and then compute evaluation results
  for e in range(epochs):

    train_loss, train_accuracy = training_step(modified_model, train_loader, optimizer, cost_function)
    torch.cuda.empty_cache()
    val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
    torch.cuda.empty_cache()
    # logs to TensorBoard
    log_values(writer, e, val_loss, val_accuracy, "Validation")

    print('Epoch: {:d}'.format(e+1))
    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')
    torch.cuda.empty_cache()
    # compute final evaluation results
    print('After training:')
    train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
    torch.cuda.empty_cache()
    val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
    torch.cuda.empty_cache()
    test_loss, test_accuracy = test_step(modified_model, test_loader, cost_function)
    torch.cuda.empty_cache()

    # log to TensorBoard
    log_values(writer, epochs, train_loss, train_accuracy, "train")
    log_values(writer, epochs, val_loss, val_accuracy, "validation")
    log_values(writer, epochs, test_loss, test_accuracy, "test")

    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
    print('-----------------------------------------------------')
    torch.cuda.empty_cache()


  # closes the logger
  writer.close()
  torch.save(modified_model.state_dict(), 'model.pth')


In [ ]:
main()

4001
999
5023
Before training:
	Training loss 1.71613, Training accuracy 3.28
	Validation loss 0.76210, Validation accuracy 17.32
	Test loss 0.71806, Test accuracy 14.03
-----------------------------------------------------


In [ ]:
def draw_corner_rect(bb, color='red'):
    bb = np.array(bb, dtype=np.float32)
    print(bb)
    return plt.Rectangle((bb[1], bb[0]), bb[3]-bb[1], bb[2]-bb[0], color=color,
                         fill=False, lw=3)

def draw_bbx(bb, color):
    plt.gca().add_patch(draw_corner_rect(bb, color))


def inference(model, query, image, gd_bbox):

    model.load_state_dict(torch.load('model.pth'))
    model.eval()

    with torch.no_grad():

      bbox = model(image.unsqueeze(0).to(device), query.unsqueeze(0).to(device))
      print('inference')
      print(bbox)
      print(bbox.shape)

      #caption = tokenizer.decode(query[0], skip_special_tokens=False)


      image = image.permute(1,2,0)
      img = cv2.cvtColor(image.numpy(), cv2.COLOR_BGR2RGB)

      out_bb = bbox.detach().cpu().numpy()
      out_bb = out_bb.astype(int)
      print(out_bb)
      print(out_bb.shape)
      plt.imshow(img)

      draw_bbx(out_bb[0],'red')

      plt.axis('off')
      plt.show()






In [ ]:
x = test_data[4]


inference(modified_model,
             query=x['captions'],
             image=x['image'],
              gd_bbox=x['bbox'])
